# Minimum working example for modeling photostimulation in Zebrafish

Data Loading

In [ ]:
import numpy as np
import pandas as pd
import photostimulation as ps
import adaptive_latents as al
from adaptive_latents import NumpyTimedDataSource, Bubblewrap, default_rwd_parameters, BWRun
from adaptive_latents.transforms.utils import prosvd_data

In [ ]:
# Specify the paths to the files
stim_file_path = '../../generated/datasets/fish/output_020424_ds1/stimmed.txt'
C_file_path = '../../generated/datasets/fish/output_020424_ds1/analysis_proc_C.txt'
photo_file_path = '../../generated/datasets/fish/output_020424_ds1/photostims.npy'

# Load the files
stim = np.loadtxt(stim_file_path) 
"""1st entry: frame number,
2nd entry: ignore,
3rd entry: angle of motion L,
4th entry: angle of motion R,
5th entry: timestamp,"""
C = np.loadtxt(C_file_path)#
"""Calcium imaging. 
1st entry is neuron ID,
2nd is time (frame)"""
photostim = np.load(photo_file_path)
"""1st entry: frame number,
2nd entry: counter of stims,
3rd entry: neuron ID,
4th entry: position X of neuron,
5th entry: position Y of neuron,"""

# extra note: Fs= 2.3 Hz

Run BW

In [ ]:
#Perform dimensionality reduction
smallC_photo = prosvd_data(input_arr=C.T,output_d=7, init_size=20, centering=True)

In [ ]:
t = np.arange(smallC_photo.shape[0])
in_ds = NumpyTimedDataSource(smallC_photo, t, time_offsets=(0,1))
bw = Bubblewrap(dim=in_ds.output_shape, **dict(default_rwd_parameters, M=100, num=1000, B_thresh=-10, num_grad_q=1))

In [ ]:
#run BW
br = BWRun(bw=bw, in_ds=in_ds)
br.run(limit=5500, save=False)

Extract the log predictive probability values

In [ ]:
predictions_list = ps.extract_predictions([br], offset=1)

Create a data frame for photostimulation and log PP rankings

In [ ]:
final_df=ps.rank_neurons(stim, C, photostim, predictions_list)

In [ ]:
#let's sort by change in log PP ranking
df_sorted = final_df.sort_values(by=['ranking', 'frame_number'])

df_sorted.iloc[:15]

Plotting trajectories with PCA and proSVD for particular neuron trials

In [ ]:
ps.plot_photostim(31,'pca',C, photostim, final_df)

In [ ]:
ps.plot_photostim(31,'proSVD',C, photostim, final_df)

Plot the Log PP of a determined neuron 

Note: I decided to limit the graph using frame numbers instead of neuron_ID for flexibility, so
you need to look in the data frame the desired start and end 

In [ ]:
#for neuron 31:
al.plotting_functions.compare_metrics([br], offset=1, red_lines_frames=photostim[:, 0] , xlim_start=3330, xlim_end=3500)

In case you want to see how BW is fittinfg the data:

WARNING: it takes a long time to render

In [ ]:
"""
#proSVD in two dimensions and run BW

smallC_photo = prosvd_data(C.T,2, 20, centering=True)
t = np.arange(smallC_photo.shape[0])
in_ds = NumpyTimedDataSource(smallC_photo, t, time_offsets=(0,1))
bw = Bubblewrap(dim=in_ds.output_shape, **dict(default_rwd_parameters, M=100, num=1000, B_thresh=-10, num_grad_q=1))
#run BW
br = BWRun(bw=bw, in_ds=in_ds)
br.run(limit=5500, save=False)

#create animation
ps.bw_animation(smallC_photo, bw.A, bw.mu, bw.L, bw.n_obs, save_path='fishBW_animation3.mp4', fps=10)"""